In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv("dataset/titanic_train.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [7]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [5]:
# step 1 --> train.test.split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                test_size=0.2,
                                                random_state=42)

In [6]:
x_train.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
497,3,male,NaN,0,0,15.1000,S
561,3,male,40.0,0,0,7.8958,S
118,1,male,24.0,0,1,247.5208,C
565,3,male,24.0,2,0,24.1500,S
547,2,male,NaN,0,0,13.8625,C
563,3,male,NaN,0,0,8.0500,S
229,3,female,NaN,3,1,25.4667,S
756,3,male,28.0,0,0,7.7958,S
6,1,male,54.0,0,0,51.8625,S
214,3,male,NaN,1,0,7.7500,Q


In [7]:
#imputation transformer

transformer_1 = ColumnTransformer([
                ('impute_age',SimpleImputer(),[2]),
                ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [8]:
# one hot encoding

transformer_2 = ColumnTransformer([
            ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [9]:
# scaling

transformer_3 = ColumnTransformer([
            ('scale',MinMaxScaler(),slice(0,10))
])

In [10]:
# Feature Selection

transformer_4 = SelectKBest(score_func=chi2,k=8)

In [11]:
# train model

transformer_5 = DecisionTreeClassifier()

# Create a Pipeline

In [18]:
pipe = Pipeline([
    ('trf1',transformer_1),
    ('trf2',transformer_2),
    ('trf3',transformer_3),
    ('trf4',transformer_4),
    ('trf5',transformer_5)
])

## Pipeline vs make_pipeline

In [13]:
# alternative syntax

pipe = make_pipeline(transformer_1,transformer_2,transformer_3,transformer_4,transformer_5)

In [19]:
# train 

pipe.fit(x_train,y_train)

C:\Users\Hari Om\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000027DC949A170>)),
                ('trf5', DecisionTreeClassifier())])

# Explore Pipeline

In [20]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [16]:
# Explore Pipeline

pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000027DC949A170>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [21]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [22]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [23]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [24]:
# predict

y_pred = pipe.predict(x_test)

In [25]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross Validation using Pipeline

In [ ]:
# cross validation using cross val_score

from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train,y_train, cv=5, scoring='accuracy').mean()

# Gridsearch using Pipeline

In [ ]:
# gridsearch

params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train,y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

# Exporting the Pipeline

In [27]:
# Export
import pickle
pickle.dump(pipe,open('titanic_model/pipe.pkl','wb'))

In [28]:
pipe = pickle.load(open('titanic_model/pipe.pkl','rb'))

In [29]:
#assume the data

## Assume user input
# Pclass/gender/age/sibsp/parch/Embarked

test_input = np.array([2,'male',31.0,0,0,10.5,'S'],dtype= object).reshape(1,7)

In [30]:
pipe.predict(test_input)

C:\Users\Hari Om\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\Hari Om\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)